##**[DACON] 악성 URL 분류 AI 경진대**

### **주제 및 데이터 설명**

**대회 설명**
- 주제: 악성 URL 분류 AI 알고리즘 개발
- 설명: URL 데이터를 활용하여 악성 URL 여부를 분류하는 AI 알고리즘 개발

**데이터**

train.csv
- ID : 샘플별 고유 ID (6,995,056 rows)
- URL
- label  [예측 목표]
  - 0 : 정상 URL
  - 1 : 악성 URL

test.csv
- ID : 샘플별 고유 ID (1,747,689 rows)
- URL

sample_submission.csv [파일] - 제출 양식
- ID : 샘플별 고유 ID
- probability : 예측한 악성 URL일 확률

### **코드 흐름**

**EDA**

1. 기본 특징 파악
- 길이, 서브 도메인 개수가 많을수록 악성 비율이 높음
2. 특수 문자 파악
- 특수 문자의 개수가 많을수록, @문자를 포함할 경우 악성 비율이 높음
3. 텍스트 파악
- 대문자를 포함하는 경우 URL이 IP주소로 구성되어 있을 경우 악성 비율이 높음
4. 구성요소 별 특징
- 서브도메인에 - 문자를 포함하는 경우 path부분의 길이가 길수록 악성 비율이 높음
5. 엔트로피 특징
- 악성 URL이 정성 URL보다 약간 더 높은 엔트로피 값을 보이며, 이는 무작위로 생성된 문자열일 가능성이 높음


**Feature Engineering & preprocessing**
1. preprocessing
- urlparse 라이브러리를 사용하여 URL을 각 구성요소로 구분
- 데이터에 프로토콜 부분은 포함되어 있지 않으므로 제외
- subdomain, tld, sld, path, params, query, fragment 처리
2. Feature Engineering

(1) 특성값 분류
- 기본 특징: length, subdomain_count, num_digit 등: URL의 길이, 숫자 개수
- 특수 문자: special_char_count, at_sign, double_slash 등: 특수문자 개수
- 텍스트: uppercase, ip_address, fishing_keyword 등: 대문자 포함 여부, IP 주소 여부
- 구성 요소 별 특징: url_tld_hyphen, url_tld_length등: 각 URL 구성요소의 길이, 특수문자의 개수 등
- 엔트로피: entropy_URL, entropy_tld 등: URL과 각 구성요소의 엔트로피

(2) 인코딩
- BinaryEncoder 사용
  - OneHotEncoder을 사용하면 피처의 수가 너무 많아지는 단점 발생
- Catboost 모델에서는 인코딩을 진행하지 않고 사용

(3) 스케일링
- StandardScaler 사용, RobustScaler 사용
- 다른 스케일러를 사용한 경우 유의미한 성능변화가 나타나지 않았음

**전반적인 모델 흐름**

1. XGBoost, LightGBM, CatBoost 등의 모델을 개별적으로 활용하여 1차 모델 성능 올림 + 각 모델의 앙상블을 이용한 성능향상 수행

- XGBoost, CatBoost, LightGBM의 트리기반 모델을 사용
  - 피처 기반 정형 데이터에는 트리기반 머신러닝 모델이 적합할 것이라고 생각했기 때문
- 데이터 샘플이 너무 많았기 때문에 컴퓨터 자원 부족으로 인한 딥러닝 수행 불가
- GPU자원을 사용하지 못하는 로지스틱 회귀, 랜덤 포레스트 모델의 경우 학습에 너무 많은 시간을 소요하므로 제외

2. Optuna를 이용한 하이퍼 파라미터 튜닝
- 트리 기반 모델을 최적화하는 경우 매우 효과적인 튜닝법

3. 앙상블
- SoftVoting: 서로 다른 트리 기반 모델을 앙상블 하는 경우 softvoting을 수행하면 일반화 성능이 더욱 향상되는 점을 응용
- logit 반환: 확률값이 너무 치우치는 것을 방지하기 위하여 softvoting의 극단적인 확률을 완화하고 모델 간 차이를 더욱 명확하게 반영함

**성능향상을 위한 코드의 차별점**
1. GPU의 사용
- 이전에도 우승자 코드에서 나왔던 것과 비슷한 맥락으로 좋은 예측 성능을 확보하기 위해 GPU가 실행되는 모델을 사용하는 전략을 수행하였음
(GPU가 돌아가지 않는 모델은 과감히 버리는 전략을 가져감)

2. SoftVoting + logit반환을 동시에 사용
- softvoting에서 발생될 수 있는 극단적인 값을 logit반환을 이용하여 해당 확률을 완화함과 동시에 모델 간의 차이를 명확하게 반영하여 각 모델의 이점을 살리는 방향으로 수행

3. 꼼꼼한 feature engineering
- 타 우승자 코드에서도 느꼈던 점이지만, 예측성능을 효과적으로 높이기 위해선 먼저 데이터를 완벽하게 파악하여 어떠한 이상치, 결측치가 있는지 확인하고, 데이터를 어떻게 가공할 것인지를 결정하는 것이 중요하다는 것을 다시한번 느낌

